In [11]:
#si nltk non installé
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [ ]:
import csv as csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from sklearn import linear_model, datasets
import random
from nltk.stem.snowball import FrenchStemmer
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
import nltk.data

#importation des fichiers CSV
dfMotEmo = pd.read_csv('motEmo.csv',encoding="utf-8-sig",delimiter=";",header=0)
dfEmoCat = pd.read_csv('emoCat.csv',encoding="utf-8-sig",delimiter=";",header=0)


#création du model d'apprentissage
table= dfMotEmo[['expression','categorie']].as_matrix()
labeled_names = ([({'nom': name[0]}, name[1]) for name in table])
random.shuffle(labeled_names)
classifier = nltk.NaiveBayesClassifier.train(labeled_names)

#stemming pour du francais
stemmer = FrenchStemmer()
french_stopwords = set(stopwords.words('french'))
tokenizer = WordPunctTokenizer()
tokenizerPhrase = nltk.data.load('tokenizers/punkt/french.pickle')

#nombre de ligne dans la table dfMotEmo
compteur=len(dfMotEmo.index)

def nettoyageEtRechercheDonnee(texte,donnee):
    split=texte.split()
    for mot in split:
        #nettoyage des données: j'enlève les ponctuations
        ponctuation=['.',',',';',':','!','?',')','(','\'','"']
        for p in ponctuation:
            if mot[0]==p:
                mot=mot[1:]
            if mot[-1]==p:
                mot=mot[:-1]
        #nettoyage des données: j'enleve donneeles genres sur un mot pour qu'il soit reconnu dans la BDD
        if mot[-3:]=="ées" or mot[-3:]=="és" :
            mot=mot[:-3]+"é"
        if mot[-1:]=="s":
            mot=mot[:-1]
        #recherche dans la table des expression par emotion
        for expr in donnee['expression']:
            if stemmer.stem(expr)==stemmer.stem(mot):
                id=donnee[donnee['expression']==expr]['id']
                res.append([expr,id])
    #si c'est une expression de plusieurs mots
    textLessPron=tokenizer.tokenize(texte)
    textLessPron = [token for token in textLessPron if token.lower() not in french_stopwords]
    for expr in donnee['expression']:
        if len(expr.split()) > 1:
            #on enleve les pronoms
            exprLessPron=tokenizer.tokenize(expr)
            exprLessPron = [token for token in exprLessPron if token.lower() not in french_stopwords] 
            #on reconstruit le texte et l'expression nettoyés
            str1 = ' '.join(textLessPron)
            str2 = ' '.join(exprLessPron)
            #if texte.find(expr) != -1:
            if str1.find(str2) != -1:
                print(expr)
                cat=donnee[donnee['expression']==expr]['id']
                res.append([expr,id])
    print(res)
    return res;

def prediction(donnee):
    tabResultat=[]
    for r in donnee:
        #cat=logreg.predict(r[1])
        cat=classifier.classify({'nom':r[0]})
        print(cat)
        print(dfEmoCat[dfEmoCat['categorie']==cat]['emotion'])
        tabResultat.append(cat)
    #compte le nombre d'occurence pour chaque émotions trouvées
    print('table des id des émotions trouvées: ',tabResultat)
    compte = {}.fromkeys(set(tabResultat),0)
    for valeur in tabResultat:
        compte[valeur] += 1
    #prends l'occurence la plus haute
    n1=0
    for valeur in tabResultat:
        if compte[valeur] > n1:
            resu=valeur
            n1=compte[valeur]
    print('id de l\'emotion retenu (la plus grande occurence) :', resu)
    resu=dfEmoCat[dfEmoCat['categorie']==resu]['emotion'].values
    return resu[0];

def apprentAuto(texte,compteur,tokenizer,donnee):
    # chargement du tokenizer
    tokens = tokenizer.tokenize(texte)
    for phrase in tokens:
        res=nettoyageEtRechercheDonnee(phrase,donnee)
        resu=prediction(res)
        categorie=dfEmoCat[dfEmoCat['emotion']==resu]['categorie'].values
        compteur=compteur+1
        fields=[compteur,phrase,categorie[0]]
        #ouverture du fichier
        with open(r'motEmo.csv', 'a',newline="\n") as f:
            writer = csv.writer(f,delimiter=';')
            #ajout d'une nouvelle ligne
            writer.writerow(fields)
    return compteur;
    
    
#programme principal
rest=True;
while(rest):
    print('tapez 1 pour lancer le test')
    print('tapez 2 pour ajouter un mot ou une expression associé à une émotion à la base de donnée')
    print('tapez "q" pour quitter l\'application')
    inp=input('tapez la commande:')
    
    if inp=='1':
        
        texte=input("Entrez la texte à tester : ");
        res=[]
        #nettoyage des données
        tableDonnee=nettoyageEtRechercheDonnee(texte,dfMotEmo)
        #prediction pour chaque mot et expression trouvés
        if tableDonnee == []:
            resultat='neutre'
        else:
            resultat=prediction(tableDonnee)
        print('---------------RESULTAT----------------')
        print(resultat)
        print('---------------RESULTAT----------------')
        print('')
        print('')
        #apprentissage auto
        if tableDonnee != []:
            compteur=apprentAuto(texte,compteur,tokenizerPhrase,dfMotEmo);
        
    if inp=='2':
            compteur=compteur+1
            exprAsave=input("Entrez l'expression ou le mot : ");
            print('table des émotions:')
            print('tranquilité = 1, Surprise=2, Joie=3, Tristesse=4, Dégoût=5, Colère=6, Fureur=7, Peur=8, Terreur=9, Coupure avec ses émotions=10')
            catAsave=input("Entrez le numéro de l\'émotion : ");
            fields=[compteur,exprAsave,catAsave]
            #ouverture du fichier
            with open(r'motEmo.csv', 'a',newline="\n") as f:
                writer = csv.writer(f,delimiter=';')
                #ajout d'une nouvelle ligne
                writer.writerow(fields)
            print('')
            print('')
            
            
    if inp=='q':
        rest=False
        print('fin du programme')




tapez 1 pour lancer le test
tapez 2 pour ajouter un mot ou une expression associé à une émotion à la base de donnée
tapez "q" pour quitter l'application
tapez la commande:1
Entrez la texte à tester : l'amour de la vie
[]
---------------RESULTAT----------------
neutre
---------------RESULTAT----------------


tapez 1 pour lancer le test
tapez 2 pour ajouter un mot ou une expression associé à une émotion à la base de donnée
tapez "q" pour quitter l'application
tapez la commande:amour de la vie
tapez 1 pour lancer le test
tapez 2 pour ajouter un mot ou une expression associé à une émotion à la base de donnée
tapez "q" pour quitter l'application
tapez la commande:1
Entrez la texte à tester : amour de la vie
[['amoureux', 241    242
Name: id, dtype: int64], ['amour', 242    243
Name: id, dtype: int64]]
3.0
2    Joie
Name: emotion, dtype: object
3.0
2    Joie
Name: emotion, dtype: object
table des id des émotions trouvées:  [3.0, 3.0]
id de l'emotion retenu (la plus grande occurence) : 3.0
-